In [5]:
using Pkg
using DataFrames
using Colors
using Images
using ImageView
using DecisionTree
using CSV

In [6]:
function read_data(typeData, labelsInfo, imageSize, path)
 #Intialize x matrix
 x = zeros(size(labelsInfo, 1), imageSize)

 #for (index, idImage) in enumerate(labelsInfo["ID"])
 for (index, idImage) in enumerate(labelsInfo[:ID])
  #Read image file
  nameFile = "$(path)/$(typeData)Resized/$(idImage).Bmp"
  img = load(nameFile)

  # Convert images to grayscale
  temp = Gray.(img)
  #imshow(temp)  # Show all images; not recommended for large

  #Transform image matrix to a vector and store
  #it in data matrix
  x[index, :] = reshape(temp, 1, imageSize)
 end
 return x
end

read_data (generic function with 1 method)

In [11]:
## Main programme
imageSize = 400 # 20 x 20 pixel

#Set location of data files, folders
path =  "C:\\Users\\RAVISINGH\\Desktop\\Data Science\\julia"


#Read information about training data , IDs.
labelsInfoTrain = CSV.read("$(path)\\trainLabels.csv")

#Read training matrix
xTrain = read_data("train", labelsInfoTrain, imageSize, path)

#Read information about test data ( IDs ).
labelsInfoTest = CSV.read("$(path)\\sampleSubmission.csv")

#Read test matrix
xTest = read_data("test", labelsInfoTest, imageSize, path)

#Get only first character of string (convert from string to character).
#Apply the function to each element of the column "Class"
yTrain = Int.(map(x -> x[1], labelsInfoTrain[:Class]))

#Convert to array
yTrain = convert(Array, yTrain)


6283-element Array{Int64,1}:
 110
  56
  84
  73
  82
  87
  76
 108
  75
  71
  65
  65
  67
   ⋮
 116
  83
 101
 108
  79
  82
  76
  78
  73
  80
  78
  82

In [39]:

model = build_forest(yTrain, xTrain, 20, 150, 0.8)



Ensemble of Decision Trees
Trees:      150
Avg Leaves: 1927.96
Avg Depth:  18.466666666666665

In [40]:
#Get predictions for test data
predTest = apply_forest(model, xTest)

#Convert integer predictions to character
labelsInfoTest[:Class] = Char.(predTest)


#accuracy = nfoldCV_forest(yTrain, xTrain, 20, 50, 4, 1.0);
#println("4 fold accuracy: $(mean(accuracy))")

#Save predictions
#CSV.write("$(path)\\juliaSubmission.csv", labelsInfoTest, separator=',', header=true)
labelsInfoTest


,ID,Class
,Int64,Char
1,6284,'H'
2,6285,'E'
3,6286,'A'
4,6287,'D'
5,6288,'E'
6,6289,'I'
7,6290,'0'
8,6291,'a'
9,6292,'S'


### KNN

In [46]:
xTrain = xTrain'
xTest = xTest'

400×6220 LinearAlgebra.Adjoint{Float64,Array{Float64,2}}:
 0.45098   0.282353  0.113725   0.592157  …  0.329412  0.635294  0.403922
 0.439216  0.27451   0.117647   0.592157     0.341176  0.67451   0.411765
 0.443137  0.278431  0.0980392  0.584314     0.352941  0.678431  0.407843
 0.439216  0.290196  0.0901961  0.6          0.356863  0.619608  0.403922
 0.447059  0.329412  0.105882   0.572549     0.560784  0.639216  0.415686
 0.45098   0.356863  0.101961   0.576471  …  0.670588  0.596078  0.419608
 0.447059  0.380392  0.0901961  0.611765     0.701961  0.611765  0.411765
 0.443137  0.372549  0.0980392  0.596078     0.729412  0.639216  0.411765
 0.439216  0.286275  0.0901961  0.596078     0.780392  0.592157  0.411765
 0.439216  0.266667  0.117647   0.596078     0.780392  0.619608  0.423529
 0.45098   0.27451   0.117647   0.603922  …  0.796078  0.627451  0.419608
 0.439216  0.278431  0.0862745  0.6          0.807843  0.627451  0.423529
 0.45098   0.278431  0.0901961  0.592157     0.807843 

In [42]:
function euclidean_distance(a, b)
 distance = 0.0 
 for index in 1:size(a, 1) 
  distance += (a[index]-b[index]) * (a[index]-b[index])
 end
 return distance
end

euclidean_distance (generic function with 1 method)

In [43]:
#This function finds the k nearest neighbors of the i data point.
function get_k_nearest_neighbors(x, i, k)

 nRows, nCols = size(x)

 #Let's initialize a vector image_i. We do this so that 
 #the image i is accessed only once from the main X matrix.
 #The program saves time because no repeated work is done.
 #Also, creating an empty vector and filling it with each 
 #element at a time is faster than copying the entire vector at once.
 #Creating empty array (vector) of nRows elements of type Float32(decimal)
 imageI = Array(Float32, nRows) 

 for index in 1:nRows
  imageI[index] = x[index, i]
 end

 #For the same previous reasons, we initialize an empty vector 
 #that will contain the j data point
 imageJ = Array(Float32, nRows)
 
 #Let's also initialize an empty vector that will contain the distances
 #between the i data point and each data point in the X matrix.
 distances = Array(Float32, nCols)
 
 for j in 1:nCols
  #The next for loop fills the vector image_j with the j data point 
  #from the main matrix. Copying element one by one is faster
  #than copying the entire vector at once.
  for index in 1:nRows
   imageJ[index] = x[index, j]
  end
  #Let's calculate the distance and save the result
  distances[j] = euclidean_distance(imageI, imageJ)
 end
 
 #The following line gives the indices sorted by distances.
 sortedNeighbors = sortperm(distances)
 
 #Let's select the k nearest neighbors. We start with the 
 #second closest. See explanation below.
 kNearestNeighbors = sortedNeighbors[2:k+1]
 return kNearestNeighbors
end

get_k_nearest_neighbors (generic function with 1 method)

In [44]:
#This function assigns a label to the i point according to
#the labels of the k nearest neighbors. The training
#data is stored in the X matrix, and its labels are stored in y.

function assign_label(x, y, k, i)
 kNearestNeighbors = get_k_nearest_neighbors(x, i, k)
 
 #let's make a dictionary to save the counts of 
 #the labels
 # Dict{}() is also right .
 # Int,Int indicates the dictionary to expect integer values 
 counts = Dict{Int, Int}() 

 #The next two variables keep track of the 
 #label with the highest count.
 highestCount = 0
 mostPopularLabel = 0

 #Iterating over the labels of the k nearest neighbors
 for n in kNearestNeighbors
  labelOfN = y[n]
  #Adding the current label to our dictionary
  #if it's not already there
  if !haskey(counts, labelOfN)
   counts[labelOfN] = 0
  end
  #Add one to the count
  counts[labelOfN] += 1 

  if counts[labelOfN] > highestCount
   highestCount = counts[labelOfN]
   mostPopularLabel = labelOfN
  end 
 end
 return mostPopularLabel
end

assign_label (generic function with 1 method)

In [47]:
#In this example, we use a value of 1 for k.
k=1 
yPredictions = [assign_label(xTrain, yTrain, k, i) for i in 1:size(xTrain, 2)]

MethodError: MethodError: no method matching Array(::Type{Float32}, ::Int64)
Closest candidates are:
  Array(!Matched::LinearAlgebra.UniformScaling, ::Integer, !Matched::Integer) at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\uniformscaling.jl:329